In [5]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS PhoneBook (
    id SERIAL PRIMARY KEY,
    name TEXT,
    phone TEXT
);
""")

cur.execute("""
CREATE OR REPLACE FUNCTION search_phonebook_start(pattern TEXT)
RETURNS TABLE(id INT, name TEXT, phone TEXT)
AS $$
BEGIN
    RETURN QUERY
    SELECT * FROM PhoneBook
    WHERE PhoneBook.name ILIKE pattern || '%';
END;
$$ LANGUAGE plpgsql;
""")

cur.execute("INSERT INTO PhoneBook (name, phone) VALUES (%s, %s) ON CONFLICT DO NOTHING;", ('David', '123123'))
cur.execute("INSERT INTO PhoneBook (name, phone) VALUES (%s, %s) ON CONFLICT DO NOTHING;", ('Daniel', '456456'))
cur.execute("INSERT INTO PhoneBook (name, phone) VALUES (%s, %s) ON CONFLICT DO NOTHING;", ('Alice', '789789'))
conn.commit()

cur.execute("SELECT * FROM search_phonebook_start(%s);", ('D',))
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()

(7, 'David', '556823')
(10, 'David', '123123')
(11, 'Daniel', '456456')
(13, 'David', '123123')
(14, 'Daniel', '456456')


In [6]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS PhoneBook (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE,
    phone TEXT
);
""")

cur.execute("""
CREATE OR REPLACE PROCEDURE insert_or_update_user(username TEXT, userphone TEXT)
LANGUAGE plpgsql
AS $$
BEGIN
    IF EXISTS (SELECT 1 FROM PhoneBook WHERE name = username) THEN
        UPDATE PhoneBook SET phone = userphone WHERE name = username;
    ELSE
        INSERT INTO PhoneBook(name, phone) VALUES (username, userphone);
    END IF;
END;
$$;
""")

cur.execute("CALL insert_or_update_user(%s, %s);", ('David', '111111'))
cur.execute("CALL insert_or_update_user(%s, %s);", ('Alice', '999999'))
conn.commit()

cur.execute("SELECT * FROM PhoneBook;")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()

(8, 'Bob', '12345')
(11, 'Daniel', '456456')
(14, 'Daniel', '456456')
(7, 'David', '111111')
(10, 'David', '111111')
(13, 'David', '111111')
(1, 'Alice', '999999')
(9, 'Alice', '999999')
(12, 'Alice', '999999')
(15, 'Alice', '999999')


In [7]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("DROP PROCEDURE IF EXISTS insert_users(IN names TEXT[], IN phones TEXT[]);")

create_proc = '''
CREATE OR REPLACE PROCEDURE insert_users(IN names TEXT[], IN phones TEXT[])
LANGUAGE plpgsql
AS $$
DECLARE
    i INT;
    incorrect_names TEXT[] := ARRAY[]::TEXT[];
    incorrect_phones TEXT[] := ARRAY[]::TEXT[];
BEGIN
    FOR i IN 1..array_length(names, 1) LOOP
        IF phones[i] ~ '^[0-9]{10,15}$' THEN
            INSERT INTO PhoneBook(name, phone) VALUES (names[i], phones[i]);
        ELSE
            incorrect_names := array_append(incorrect_names, names[i]);
            incorrect_phones := array_append(incorrect_phones, phones[i]);
        END IF;
    END LOOP;

    RAISE NOTICE 'Некорректные данные:';
    FOR i IN 1..array_length(incorrect_names, 1) LOOP
        RAISE NOTICE 'Имя: %, Телефон: %', incorrect_names[i], incorrect_phones[i];
    END LOOP;
END;
$$;
'''

cur.execute(create_proc)
conn.commit()
cur.close()
conn.close()

Calling the procedure

In [8]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()
names = ['Alice', 'Bob', 'Charlie']
phones = ['87001112233', 'abc123', '87007778899']

cur.execute("CALL insert_users(%s, %s);", (names, phones))

conn.commit()
cur.close()
conn.close()

Checking the table`s data

In [9]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("SELECT * FROM PhoneBook;")
rows = cur.fetchall()

for row in rows:
    print(row)

cur.close()
conn.close()

(8, 'Bob', '12345')
(11, 'Daniel', '456456')
(14, 'Daniel', '456456')
(7, 'David', '111111')
(10, 'David', '111111')
(13, 'David', '111111')
(1, 'Alice', '999999')
(9, 'Alice', '999999')
(12, 'Alice', '999999')
(15, 'Alice', '999999')
(16, 'Alice', '87001112233')
(17, 'Charlie', '87007778899')


In [10]:
import psycopg2

def fetch_paginated_data(limit, offset):
    conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)
    cur = conn.cursor()
    cur.execute("SELECT * FROM PhoneBook ORDER BY id LIMIT %s OFFSET %s;", (limit, offset))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return rows

In [11]:
data = fetch_paginated_data(2, 0)
for row in data:
    print(row)

(1, 'Alice', '999999')
(7, 'David', '111111')


In [12]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("DROP PROCEDURE IF EXISTS delete_user(IN user_name TEXT, IN user_phone TEXT);")

create_proc = '''
CREATE OR REPLACE PROCEDURE delete_user(IN user_name TEXT, IN user_phone TEXT)
LANGUAGE plpgsql
AS $$
BEGIN
    DELETE FROM PhoneBook
    WHERE name = user_name OR phone = user_phone;
END;
$$;
'''

cur.execute(create_proc)
conn.commit()
cur.close()
conn.close()

Calling the procedure

In [13]:
conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)
cur = conn.cursor()

cur.execute("CALL delete_user(%s, %s);", ('Alice', '87007778899'))

conn.commit()
cur.close()
conn.close()

Checking the table`s data

In [14]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute("SELECT * FROM PhoneBook;")
rows = cur.fetchall()

for row in rows:
    print(row)

cur.close()
conn.close()

(8, 'Bob', '12345')
(11, 'Daniel', '456456')
(14, 'Daniel', '456456')
(7, 'David', '111111')
(10, 'David', '111111')
(13, 'David', '111111')
